In [1]:
import sys
sys.path.append("../src")


In [2]:
from ddd import Tag

In [3]:
import pandas as pd

In [ ]:
def load_data(as_df: bool = False):
    data = pd.read_csv("../data/bewertungen.csv")

    # convert to objects
    if not as_df:
        data = [Tag.from_dataframe(row) for _, row in data.iterrows()] 

    return data

In [13]:
data = load_data()
#data

In [14]:
tag = data[5]
tag

Tag(WochenID=2, FolgenID=205, Datum='18.12.2006', Ort=None, K1=None, K2=9, K3=8, K4=7, K5=9, Summe=33, Mikkel=8, Andi=7, Chat=None, Person='Marco', Bemerkung='Chat: keine Wertung abgegeben')